#### Import packages

In [1]:
import numpy as np
import pandas as pd
from pygmid import Lookup as lk

#### Technology data

In [2]:
n = lk('../nfet_03v3.mat')
p = lk('../pfet_03v3.mat')

#### Specifications

In [3]:
GBW=100e6; CL=1e-12

#### Design choices

In [4]:
gm_id0 = 12
gm_id1 = 12
gm_id2 = 12
l0 = 0.5
l1 = 0.5
l2 = 0.5

#### Sizing and benchmarking

In [5]:
# calculate gm of differential pair
gm1 = 2*np.pi*GBW*CL

# size all transistors
id1 = gm1/gm_id1; id2=id1; id0=2*id1
jd1 = n.lookup('ID_W', GM_ID=gm_id1, L=l1)
w1 = id1/jd1
jd2 = p.lookup('ID_W', GM_ID=gm_id2, L=l2)
w2 = id1/jd2
w0 = 2*w1

# estimate mirror pole
cgg2 = w2*p.lookup('CGG_W', GM_ID=gm_id2, L=l2)
cdd2 = w2*p.lookup('CDD_W', GM_ID=gm_id2, L=l2)
cdd1 = w2*n.lookup('CDD_W', GM_ID=gm_id1, L=l1)
gm2 = gm1
fp2 = gm2/(2*cgg2+cdd1+cdd2)/(2*np.pi)

# estimate phase margin (mirror pole, LHP zero, RHP zero)
phip2 = -np.arctan(GBW/fp2)*180/np.pi
fz2 = 2*fp2
phiz2 = +np.arctan(GBW/fz2)*180/np.pi
cgd1 = w1*n.lookup('CGD_W', GM_ID=gm_id1, L=l1)
fz3 = gm1/cgd1/(2*np.pi)
phiz3 = -np.arctan(GBW/fz3)*180/np.pi
PM = 90 +phip2 +phiz2 +phiz3 

# calculate the gain
gm_gds1 = n.lookup('GM_GDS', GM_ID=gm_id1, L=l1)
gds1 = gm1/gm_gds1
gm_gds2 = p.lookup('GM_GDS', GM_ID=gm_id2, L=l2)
gds2 = gm2/gm_gds2
A0 = gm1/(gds1 + gds2)


df = pd.DataFrame( [id1/1e-6, gm1/1e-3, A0, fp2, PM], \
                   ['id1 (uA)', 'gm1 (mS)', 'A0 (V/V)', 'fp2 (Hz)', 'PM (deg)'], columns=['Value']); 
pd.set_option('display.float_format', '{:.4e}'.format); df

,Value
id1 (uA),5.2360e+01
gm1 (mS),6.2832e-01
A0 (V/V),1.0938e+02
fp2 (Hz),2.8143e+08
PM (deg),8.0359e+01


In [6]:
# finger the devices
wfing = 5
nf0 = 1+np.floor_divide(w0, wfing)
nf1 = 1+np.floor_divide(w1, wfing)
nf2 = 1+np.floor_divide(w2, wfing)
df = pd.DataFrame( [(id0*1e6, id1*1e6, id2*1e6), (w0, w1, w2), (l0, l1, l2), (nf0, nf1, nf2)], \
                   ['ID (uA)', 'w (um)', 'l (um)', 'nf'], columns=['M0', 'M1', 'M2']); 
pd.set_option('display.float_format', '{:.2e}'.format); df

,M0,M1,M2
ID (uA),1.05e+02,5.24e+01,5.24e+01
w (um),3.85e+01,1.92e+01,6.65e+01
l (um),5.00e-01,5.00e-01,5.00e-01
nf,8.00e+00,4.00e+00,1.40e+01


#### Write spice include file

In [7]:
with open('sizing_ota-5t_gf.param', 'w') as file:
    file.write(".param ibn = " + "{:.2e}".format(id0) + '\n')
    file.write(".param W0 = " + "{:.2e}".format(w0*1e-6) + '\n')
    file.write(".param W1 = " + "{:.2e}".format(w1*1e-6) + '\n')
    file.write(".param W2 = " + "{:.2e}".format(w2*1e-6) + '\n')
    file.write(".param L0 = " + "{:.2e}".format(l0*1e-6) + '\n')
    file.write(".param L1 = " + "{:.2e}".format(l1*1e-6) + '\n')
    file.write(".param L2 = " + "{:.2e}".format(l2*1e-6) + '\n')
    file.write(".param nf0 = " + "{}".format(nf0) + '\n')
    file.write(".param nf1 = " + "{}".format(nf1) + '\n')
    file.write(".param nf2 = " + "{}".format(nf2) + '\n')

#### A quick sanity check

In [8]:
CLtot = CL+ cdd1 + cdd2
print(f'CLtot = {CLtot*1e12:0.2f} pF' )

CLtot = 1.11 pF
